# Machine Learning (2h)

Voici les ventes immobilières officielles du premier semestre 2024.

## Régression

Votre mission, si toutefois vous l'acceptez, sera de réaliser des prédictions de valeurs immobilières (`Valeur fonciere`), en fonction du type de bien (`Type local` - **ne travailler que sur les maisons et les apparetements**), de la surface (`Surface reelle bati`), du nombre de pièces (`Nombre pieces principales`) et du  numéro de département (`Code departement`).

In [38]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [39]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  return X_encoded, SN

In [40]:
df = pd.read_csv('valeursfoncieres-2024-s1.txt.zip',
                 sep='|',                # Séparateur |
                 encoding='latin-1',     # Encodage pour les caractères français
                 compression='zip',      # Spécifie que c'est un fichier zip
                 low_memory=False)       # Pour éviter les warnings de memory

In [41]:
df = df.drop(['Identifiant de document', 'Reference document', '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'Identifiant local'], axis = 1)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566643 entries, 0 to 1566642
Data columns (total 35 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   No disposition              1566643 non-null  int64  
 1   Date mutation               1566643 non-null  object 
 2   Nature mutation             1566643 non-null  object 
 3   Valeur fonciere             1549589 non-null  object 
 4   No voie                     946773 non-null   float64
 5   B/T/Q                       67087 non-null    object 
 6   Type de voie                912892 non-null   object 
 7   Code voie                   1558442 non-null  object 
 8   Voie                        1558411 non-null  object 
 9   Code postal                 1558158 non-null  float64
 10  Commune                     1566643 non-null  object 
 11  Code departement            1566643 non-null  object 
 12  Code commune                1566643 non-null  int64  
 1

In [43]:
df_ml = df[['Valeur fonciere', 'Type local', 'Code departement', 'Surface reelle bati', 'Nombre pieces principales']]
df_ml = df_ml[(df_ml['Type local'] == 'Maison') | (df_ml['Type local'] == 'Appartement')]
df_ml = df_ml[df_ml['Nombre pieces principales'].isna() == False]

df_ml['Valeur fonciere'] = df_ml['Valeur fonciere'].astype(str)
df_ml['Valeur fonciere'] = df_ml['Valeur fonciere'].apply(lambda x : x.replace(',', '.'))

df_ml['Valeur fonciere'] = df_ml['Valeur fonciere'].astype(float)

df_ml_train = df_ml[df_ml['Valeur fonciere'].isna() == False]
df_ml_predict = df_ml[df_ml['Valeur fonciere'].isna() == True]

In [44]:
y = df_ml_train['Valeur fonciere']
X = df_ml_train.drop(['Valeur fonciere'], axis = 1)

X_encoded, SN = encodage_X(X)

models = [
    LinearRegression()
]

param_grids = [
    {'n_jobs': [2, 5, 8, 10],
     'fit_intercept': [True, False]}
]

df_reponse = pd.DataFrame(index = [1], columns = ['Model', 'Score', 'Paramètres'])
df_reponse['Model'] = models

for model, param_grid in zip(models, param_grids):
    gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    gs.fit(X_encoded, y)
    df_reponse.loc[df_reponse['Model'] == model,'Score'] = f"{gs.best_score_:.3f}"
    df_reponse.loc[df_reponse['Model'] == model,'Paramètres'] = f"{gs.best_params_}"

df_reponse

,Model,Score,Paramètres
1,LinearRegression(),-16409668150699770249216.000,"{'fit_intercept': False, 'n_jobs': 2}"


In [45]:
y = df_ml_train['Valeur fonciere']
X = df_ml_train.drop(['Valeur fonciere'], axis = 1)

X_encoded_train, SN = encodage_X(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded_train, y, train_size=0.75, random_state=13)

model = LinearRegression(fit_intercept = False, n_jobs = 2).fit(X_train, y_train)

X = df_ml_predict.drop(['Valeur fonciere'], axis = 1)

X_encoded_predict, SN = encodage_X(X)

colonnes = []

for element in X_encoded.columns:
    if element not in X_encoded_predict.columns:
        colonnes.append(element)


for element in colonnes:
    X_encoded_predict[element] = False

len(X_encoded.columns), len(X_encoded_predict.columns)
X_encoded_predict.columns = X_encoded.columns

df_ml_predict['Valeur fonciere'] = model.predict(X_encoded_predict)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1211/4176572390.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml_predict['Valeur fonciere'] = model.predict(X_encoded_predict)


In [46]:
df_ml_predict

,Valeur fonciere,Type local,Code departement,Surface reelle bati,Nombre pieces principales
8485,725760.0,Appartement,01,99.0,4.0
8487,725760.0,Appartement,01,99.0,4.0
14931,220432.0,Maison,01,87.0,4.0
18839,580608.0,Maison,02,126.0,3.0
20411,782080.0,Appartement,02,109.0,4.0
...,...,...,...,...,...
1551445,163280.0,Appartement,75,28.0,1.0
1555957,613120.0,Appartement,75,81.0,3.0
1556002,603584.0,Appartement,75,80.0,3.0
1560515,335792.0,Appartement,75,49.0,2.0


## Classification

Ce dataset contient des informations sur les clients d'une entreprise de télécommunications, incluant leurs caractéristiques d'utilisation du service et si oui ou non ils ont quitté l'entreprise (churn).

Votre mission sera de développer un modèle de machine learning capable de prédire si un client est susceptible de résilier son abonnement (churn) en se basant sur ses caractéristiques et son historique

### Variables du Dataset

#### Informations Démographiques et Compte
- `state` : État américain du client (catégoriel)
- `account_length` : Durée de l'abonnement en jours (numérique)
- `area_code` : Indicatif régional (catégoriel : area_code_415, area_code_408, area_code_510)
- `international_plan` : Abonnement au forfait international (binaire : yes/no)
- `voice_mail_plan` : Abonnement à la messagerie vocale (binaire : yes/no)
- `number_vmail_messages` : Nombre de messages vocaux (numérique)

#### Utilisation en Journée
- `total_day_minutes` : Minutes d'appel en journée (numérique)
- `total_day_calls` : Nombre d'appels en journée (numérique)
- `total_day_charge` : Montant facturé pour les appels en journée (numérique)

#### Utilisation en Soirée
- `total_eve_minutes` : Minutes d'appel en soirée (numérique)
- `total_eve_calls` : Nombre d'appels en soirée (numérique)
- `total_eve_charge` : Montant facturé pour les appels en soirée (numérique)

#### Utilisation de Nuit
- `total_night_minutes` : Minutes d'appel de nuit (numérique)
- `total_night_calls` : Nombre d'appels de nuit (numérique)
- `total_night_charge` : Montant facturé pour les appels de nuit (numérique)

#### Utilisation Internationale
- `total_intl_minutes` : Minutes d'appel internationaux (numérique)
- `total_intl_calls` : Nombre d'appels internationaux (numérique)
- `total_intl_charge` : Montant facturé pour les appels internationaux (numérique)

#### Service Client et Variable Cible
- `number_customer_service_calls` : Nombre d'appels au service client (numérique)
- `churn` : Variable cible indiquant si le client a résilié son abonnement (binaire : yes/no)

**Vous êtes libres d'utiliser le modèle que vous souhaitez**

In [47]:
df = pd.read_csv('http://cambon.pro/churn.csv')

In [48]:
df['international_plan'] = df['international_plan'].apply(lambda x : True if x == 'yes' else False)
df['voice_mail_plan'] = df['voice_mail_plan'].apply(lambda x : True if x == 'yes' else False)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          5000 non-null   object 
 1   account_length                 5000 non-null   int64  
 2   area_code                      5000 non-null   object 
 3   international_plan             5000 non-null   bool   
 4   voice_mail_plan                5000 non-null   bool   
 5   number_vmail_messages          5000 non-null   int64  
 6   total_day_minutes              5000 non-null   float64
 7   total_day_calls                5000 non-null   int64  
 8   total_day_charge               5000 non-null   float64
 9   total_eve_minutes              5000 non-null   float64
 10  total_eve_calls                5000 non-null   int64  
 11  total_eve_charge               5000 non-null   float64
 12  total_night_minutes            5000 non-null   f

In [50]:
y = df['churn']
X = df.drop('churn', axis = 1)

In [51]:
X_encoded, SN = encodage_X(X)

models = [
    KNeighborsClassifier(),
    SVC(),
    LogisticRegression(),
]

param_grids = [
    {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'weights': ['uniform', 'distance']},
    {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10]},
    {'C' : [1.0, 1.5]}
]

df_reponse = pd.DataFrame(index = [1, 2, 3], columns = ['Model', 'Score', 'Paramètres'])
df_reponse['Model'] = models

for model, param_grid in zip(models, param_grids):
    gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    gs.fit(X_encoded, y)
    df_reponse.loc[df_reponse['Model'] == model,'Score'] = f"{gs.best_score_:.3f}"
    df_reponse.loc[df_reponse['Model'] == model,'Paramètres'] = f"{gs.best_params_}"

df_reponse

,Model,Score,Paramètres
1,KNeighborsClassifier(),0.890,"{'n_neighbors': 8, 'weights': 'distance'}"
2,SVC(),0.911,"{'C': 1, 'kernel': 'rbf'}"
3,LogisticRegression(),0.865,{'C': 1.5}


In [52]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, train_size=0.75, random_state=13)

model = SVC(C = 1, kernel = 'rbf')

model.fit(X_train, y_train)

print(f"Score d'entrainement : {model.score(X_train, y_train)}")
print(f"Score de test : {model.score(X_test, y_test)}")

Score d'entrainement : 0.9317333333333333
Score de test : 0.9184


In [53]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(data = confusion_matrix(y_true = y_test, y_pred = model.predict(X_test)),
             index = model.classes_ + " ACTUAL",
             columns = model.classes_ + " PREDICTED")

,no PREDICTED,yes PREDICTED
no ACTUAL,1079,6
yes ACTUAL,96,69
